In [174]:
import tensorflow as tf
import re

## 트레이닝 데이터 호출
f = open("LottoTrain.txt", "r")
lines = f.readlines()
f.close()
    
## 값 세팅
X = tf.placeholder(tf.float32, [None, 46])
Y = tf.placeholder(tf.float32, [None, 46])
keep_prob = tf.placeholder(tf.float32)
W1 = tf.Variable(tf.random_normal(([46, 256]), stddev=0.01))
L1 = tf.nn.relu(tf.matmul(X, W1))
L1 = tf.nn.dropout(L1, keep_prob)
W2= tf.Variable(tf.random_normal([256,256], stddev =0.01))
L2 = tf.nn.relu(tf.matmul(L1, W2))
L2 = tf.nn.dropout(L2, keep_prob)
W3= tf.Variable(tf.random_normal([256,256], stddev =0.01))
L3 = tf.nn.relu(tf.matmul(L2, W3))
L3 = tf.nn.dropout(L3, keep_prob)

W4 = tf.Variable(tf.random_normal([256, 46], stddev = 0.01))
model = tf.matmul(L3,W4)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels =Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

## 트레이닝
epochs = 10
for epoch in range(epochs):
    totalCost = 0

    for i in range(len(lines)-1):
        input = re.sub('\n','',lines[i])
        input = re.split(',', input)
        input = input[:6]
        inputs = [0]*46
        for j in input:
            inputs[int(j)] = 1

        target = re.sub('\n','',lines[i+1])
        target = re.split(',', target)
        target = target[:6]

        targets = [0]*46
        for j in target:
            targets[int(j)] = 1
        
        inputs = [inputs]
        targets = [targets]
        
        _, costVal = sess.run([optimizer, cost], feed_dict={X:inputs, Y:targets, keep_prob:1})
        totalCost += costVal
    
#     print('Epoch:', '%04d' % (epoch +1), 'Avg.cost = ', '{:.3f}'.format(totalCost))


## 테스트 데이터 호출
f = open("LottoTest.txt", "r")
lines = f.readlines()
f.close()

## 테스트
totalCnt = 0
cnt3 = 0
cnt4 = 0
cnt5 = 0
cnt6 = 0
trash = 0
for i in range(len(lines)-1):
    input = re.sub('\n','',lines[i])
    input = re.split(',', input)
    input = input[:6]

    inputs = [0]*46
    for j in input:
        inputs[int(j)] = 1
    
    target = re.sub('\n','',lines[i+1])
    target = re.split(',', target)
    target = target[:6]
    target = [int(i) for i in target]
    targets = [0]*46
    for j in target:
        targets[int(j)] = 1
    
    inputs = [inputs]
    
    result = sess.run(model, feed_dict={X:inputs, keep_prob:1})
    result = result.flatten().argsort()
    
    cnt = 0
    for j in range(1,7):
        if(targets[result[-j]] == 1):
            cnt +=1
    totalCnt += cnt
    if cnt ==3:
        cnt3 += 1
    elif cnt == 4:
        cnt4 +=1
    elif cnt == 5:
        cnt5 +=1
    elif cnt == 6:
        cnt6 +=1
    else:
        trash+=1
        
        
print('accuracy : ', float(totalCnt/(6*(len(lines)-1))))
print(cnt3, cnt4, cnt5, cnt6, trash)
## 예측
predict = [16,24,29,40,41,42]
predicts = [0]*46
for i in predict:
    predicts[i] = 1
predicts = [predicts]

# 14,27,30,31,40,42,2
result = sess.run(model, feed_dict={X:predicts,keep_prob:1})
result = result.flatten().argsort()
    
print(result)

accuracy :  0.16666666666666666
1 0 0 0 18
[ 0  9 28 34 38 11 12  4  1 13 21 42 22 24 31 41 45 39  5 29 32  8 14 44 27
 25 23 36 20  3 33 19 10 15 17 30  2 16 43 37 18 40 26  7  6 35]
